In [1]:
from FedCVL import FedCVTClient, FedCVTServer

In [2]:
# ====================== SOLAR CLIENT ======================
import socket, pickle
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


HOST, PORT = "localhost", 9000

# Load Solar data
solar_data = pd.read_csv("../data/client2_solar.csv").values

# Split train/test
indices = np.arange(len(solar_data))
train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42)
solar_train, solar_test = solar_data[train_idx], solar_data[test_idx]

# Init client
solar_client = FedCVTClient("Solar", input_dim=solar_data.shape[1], encoding_dim=16)

# Preprocess
solar_train_tensor = solar_client.preprocess_data(solar_train, fit_scaler=True)
solar_test_tensor = solar_client.preprocess_data(solar_test)

# Connect
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((HOST, PORT))

num_rounds = 5
batch_size = 32
dataset_size = len(solar_train_tensor)
num_batches = (dataset_size + batch_size - 1) // batch_size

for round_num in range(num_rounds):
    print(f"\n[SOLAR] Round {round_num+1}/{num_rounds}")
    for batch_idx in range(num_batches):
        start, end = batch_idx*batch_size, min((batch_idx+1)*batch_size, dataset_size)
        batch = solar_train_tensor[start:end]

        # Send masked encodings
        z_masked = solar_client.send_encoded_data(batch)
        sock.send(pickle.dumps(z_masked.detach().numpy()))
        print(f"[SOLAR] Sent batch {batch_idx+1}/{num_batches}")

        # Receive gradients
        grads = pickle.loads(sock.recv(65536))
        grads_tensor = torch.tensor(grads, dtype=torch.float32)

        # Update encoder
        solar_client.local_training_step(batch)
        solar_client.update_encoder(grads_tensor, batch)

print("\n[SOLAR] Training finished.")

# --- Wait for evaluation request ---
msg = pickle.loads(sock.recv(65536))
if msg == "SEND_TEST":
    print("[SOLAR] Sending test encodings to server...")
    z_test = solar_client.send_encoded_data(solar_test_tensor)
    sock.send(pickle.dumps(z_test.detach().numpy()))



[SOLAR] Round 1/5
[SOLAR] Sent batch 1/25


/media/gaurav/ubuntudata/SemThree/DML/dml_grp49/src/FedCVL.py:130: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:2487.)
  rotation_matrix = torch.qr(rotation_matrix)[0]  # Orthogonal matrix


[SOLAR] Sent batch 2/25
[SOLAR] Sent batch 3/25
[SOLAR] Sent batch 4/25
[SOLAR] Sent batch 5/25
[SOLAR] Sent batch 6/25
[SOLAR] Sent batch 7/25
[SOLAR] Sent batch 8/25
[SOLAR] Sent batch 9/25
[SOLAR] Sent batch 10/25
[SOLAR] Sent batch 11/25
[SOLAR] Sent batch 12/25
[SOLAR] Sent batch 13/25
[SOLAR] Sent batch 14/25
[SOLAR] Sent batch 15/25
[SOLAR] Sent batch 16/25
[SOLAR] Sent batch 17/25
[SOLAR] Sent batch 18/25
[SOLAR] Sent batch 19/25
[SOLAR] Sent batch 20/25
[SOLAR] Sent batch 21/25
[SOLAR] Sent batch 22/25
[SOLAR] Sent batch 23/25
[SOLAR] Sent batch 24/25
[SOLAR] Sent batch 25/25

[SOLAR] Round 2/5
[SOLAR] Sent batch 1/25
[SOLAR] Sent batch 2/25
[SOLAR] Sent batch 3/25
[SOLAR] Sent batch 4/25
[SOLAR] Sent batch 5/25
[SOLAR] Sent batch 6/25
[SOLAR] Sent batch 7/25
[SOLAR] Sent batch 8/25
[SOLAR] Sent batch 9/25
[SOLAR] Sent batch 10/25
[SOLAR] Sent batch 11/25
[SOLAR] Sent batch 12/25
[SOLAR] Sent batch 13/25
[SOLAR] Sent batch 14/25
[SOLAR] Sent batch 15/25
[SOLAR] Sent batch 16/2